In [1]:
import torch
torch.cuda.is_available()

True

In [7]:
# Developing function to identify and resolve toponyms, and detect emotions in 150 word context 
# on either side of each toponym.

# Set a global variable for my OpenAI API key so that the model can be accessed.

import os
os.environ["OPENAI_API_KEY"] = "YOUR_KEY"

# Access libraries
from openai import OpenAI
from pydantic import BaseModel
import pandas as pd
import json

# Access model with my API key

client = OpenAI()

# Bring "Instructions" txt file into memory for the response function to access

with open("openai_EmotionSpatialDetect_prompt.txt", "r", encoding="utf-8") as f:
    instructions = f.read()

# Create response function to recognize and resolve toponyms and detect emotions in 150 character context
# on either side of the toponym.

response = client.responses.create(
  model="gpt-4.1-2025-04-14",
  instructions=instructions,
  input="I know them by sight. We're becoming friends now. I'm glad not to have to travel alone. Age 16, November 1941, Le Chambon-sur-Lignon, France. I'm in Le Chambon-sur-Lignon. It's way up on a high plateau in the mountains. It's beautiful here. What an amazing difference from Gurs! I'm staying at La Guespy, which is a home for refugees.  My friend just arrived, and before she came she saw her mother for an hour that morning through the fence, and then they put them all on cattle wagons and they were deported. And now they're gone.  We are not sure, but we think they took them to Drancy.  Martin and I are working in the new carpentry shop now. I don't really like carpentry, but at least I know how to do it, and at least I am safe.",
  text={
    "format": {
      "type": "text"
    }
  },
  reasoning={},
  tools=[
    {
      "type": "function",
      "name": "identify_toponyms_and_emotions",
      "description": "Read the input text string and identify the toponyms in the text, disambiguate their locations, and perform emotion detection on surrounding context.",
      "parameters": {
        "type": "object",
        "required": [
          "input_text",
          "toponyms"
        ],
        "properties": {
          "input_text": {
            "type": "string",
            "description": "The text string from which to identify toponyms and their surrounding context."
          },
          "toponyms": {
            "type": "array",
            "description": "Array of identified toponyms, each containing properties of location details and emotional context.",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the toponym identified."
                },
                "latitude": {
                  "type": "number",
                  "description": "The latitude coordinate of the toponym."
                },
                "longitude": {
                  "type": "number",
                  "description": "The longitude coordinate of the toponym."
                },
                "emotion": {
                  "type": "string",
                  "description": "The most likely detected emotion around the toponym.",
                  "enum": [
                    "anger",
                    "disgust",
                    "fear",
                    "joy",
                    "sadness",
                    "surprise",
                    "neutral"
                  ]
                },
                "confidence_score": {
                  "type": "number",
                  "description": "The confidence score for the detected emotion, on a scale of 0 to 1."
                },
                "context": {
                  "type": "string",
                  "description": "The block of text (150 characters on either side) surrounding the toponym used for emotion detection."
                }
              },
              "required": [
                "name",
                "latitude",
                "longitude",
                "emotion",
                "confidence_score",
                "context"
              ],
              "additionalProperties": False
            }
          }
        },
        "additionalProperties": False
      },
      "strict": True
    }
  ],
  temperature=1,
  max_output_tokens=32768,
  top_p=1,
  store=True
)

In [8]:
#Take response output, convert into json format, then normalize the data that I want 
#and put into a dataframe.

output = json.loads(response.output[0].arguments)
df = pd.json_normalize(output['toponyms'], meta=['name', 'latitude', 'longitude', 'emotion', 'confidence_score', 'context'])
df

,name,latitude,longitude,emotion,confidence_score,context
0,Le Chambon-sur-Lignon,45.060810,4.302941,joy,0.91,e now. I'm glad not to have to travel alone. A...
1,France,46.603354,1.888334,neutral,0.56,"not to have to travel alone. Age 16, November ..."
2,Le Chambon-sur-Lignon,45.060810,4.302941,joy,0.93,"41, Le Chambon-sur-Lignon, France. I'm in Le C..."
3,Gurs,43.304279,-0.776430,sadness,0.98,It's beautiful here. What an amazing differenc...
4,La Guespy,45.062610,4.312660,neutral,0.77,n amazing difference from Gurs! I'm staying at...
5,Drancy,48.923421,2.445488,sadness,0.87,d then they put them all on cattle wagons and ...


In [9]:
df.to_csv("Results_ToponymsEmotions_FewSentences.csv", encoding="utf-8-sig", index=False, header=True, mode="w+")